# ManipulateDataApp

In [1]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>

In [2]:
from IPython.display import display, Javascript
import ipywidgets as widgets

class ShowOrHideCodeCells():
    """Class to make a button to show or hide code cells within a Jupyter notebook."""
    def __init__(self, cells_visable=False):
        """Variables to initialize within class."""
        self.cells_visable = cells_visable
    
    def run(self):
        """Make a button to show or hide code cells. Hide them initially."""
        btn_widget = widgets.Button(description="Show Code Cells")
        btn_widget.on_click(self._run_show_or_hide)
        display(btn_widget)
        # Initially hide cells
        self._run_hide()
        
    def _run_hide(self):
        """Hide code cells."""
        display(Javascript("$('div.input').hide();"))
    
    def _run_show_or_hide(self, btn):
        """Show or hide code cells when button is clicked."""
        if self.cells_visable:
            display(Javascript("$('div.input').hide();"))
            btn.description = "Show Code Cells"
        else:
            display(Javascript("$('div.input').show();"))
            btn.description = "Hide Code Cells"
        self.cells_visable = not self.cells_visable

In [3]:
ShowOrHideCodeCells().run()

Button(description='Show Code Cells', style=ButtonStyle())

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
import pandas as pd, numpy as np
from IPython.display import clear_output, display, Markdown
import ipywidgets as widgets
import qgrid
import plotly.graph_objs as go
import plotly.plotly as plotly
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler
from sklearn.preprocessing import QuantileTransformer, PowerTransformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.impute import SimpleImputer, MissingIndicator

class ManipulateDataAppClass():
    """Must use class in a Jupyter notebook."""
    def __init__(self):
        """Variables to initialize within class."""
        
    def run(self):
        """Display import data widgets."""
        self._import()
        
    def _import(self):
        """Setup widgets for importing data into QGrid model."""
        import_name_widget = widgets.Text(value='C:\\Users\\cunnind2\\OneDrive - kochind.com\\DOCS4LIFE\\Research and Materials\\ipywidgets\\plotly_ipywidget_notebooks\\notebooks\\data\\cars\\cars.csv',#"C:\\Users\\username\\import_data.csv",
                                          placeholder='Type something', description='Import File:',
                                          layout=widgets.Layout(width="60%"))
        import_btn_widget = widgets.Button(description='Upload Data', button_style='')
        import_btn_widget.on_click(self._run_import)
        display(widgets.HBox([import_name_widget, import_btn_widget]))
        
        self._import_name_widget = import_name_widget
        self._import_btn_widget = import_btn_widget
        
    def _run_import(self, btn):
        """Import data into QGrid model."""
        clear_output()
        self._display_reset_start_widgets()
        self._display_qgrid(self._import_name_widget.value)
        self._display_accordion_widget()
        self._export()
        
    def _display_reset_start_widgets(self):
        """Display "Reset Table" and "Start Over" widgets."""
        reset_table_widget = widgets.Button(description='Reset Table', button_style='')
        reset_table_widget.on_click(self._reset_table)
        start_over_widget = widgets.Button(description='Start Over', button_style='')
        start_over_widget.on_click(self._start_over)
        display(widgets.HBox([reset_table_widget, start_over_widget]))
        
    def _reset_table(self, btn):
        """Reset table in QGrid model."""
        self._run_import(btn)
        
    def _start_over(self, btn):
        """Start over, as if notebook just opened."""
        clear_output()
        display(widgets.HBox([self._import_name_widget, self._import_btn_widget]))
        
    def _display_qgrid(self, filepath=None, new_data=None):
        """Display QGrid model."""
        if filepath is not None:
            import_data = pd.read_csv(filepath)
        else:
            import_data = new_data
        grid_options = {# SlickGrid options
                        'fullWidthRows': True, 'syncColumnCellResize': True,
                        'forceFitColumns': False, 'defaultColumnWidth': 150,
                        'rowHeight': 28, 'enableColumnReorder': False,
                        'enableTextSelectionOnCells': True, 'editable': True,
                        'autoEdit': False, 'explicitInitialization': True,
                        # Qgrid options
                        'maxVisibleRows': 15, 'minVisibleRows': 8,
                        'sortable': True, 'filterable': True,
                        'highlightSelectedCell': True, 'highlightSelectedRow': True}
        qgrid_data = qgrid.show_grid(import_data, grid_options=grid_options)
        display(qgrid_data)
        
        self._import_data = import_data
        self._qgrid_data = qgrid_data        
        
    def _display_accordion_widget(self):
        """"""
        tab_widgets = [self._display_dataset_split_widget(),
                       self._display_explore_widgets(),
                       self._display_preprocess_widgets(),
                       self._display_model_widgets(), 
                       self._display_code_log_widget()]
        tab_widget_names = ['Split', 'Explore', 'Preprocess', 'Model', 'Code Log']
        
        accordion = widgets.Accordion()
        children = [tab for tab in tab_widgets]
        accordion.children = children
        for i, tab_name in enumerate(tab_widget_names):
            accordion.set_title(i, tab_name)
        accordion.selected_index = None
        display(accordion)
        
        self._accordion = accordion
        self._accordion_children = children
        
    def _display_dataset_split_widget(self):
        """"""
        trainset_widget = widgets.IntSlider(description='Train:', value=60, min=40, max=98)
        devset_widget = widgets.IntSlider(description='Dev:', value=20, min=0, max=(100-trainset_widget.value)/2)
        testset_widget = widgets.IntSlider(description='Test:', value=20, min=0, max=(100-trainset_widget.value)/2)
        
        def on_train_change(change):
            devset_widget.max = (100-change['new'])/2
            testset_widget.max = (100-change['new'])/2
        trainset_widget.observe(on_train_change, names="value")
        
        return widgets.VBox([trainset_widget, devset_widget, testset_widget])
        
    def _display_explore_widgets(self):
        """"""
        xaxis_widget = widgets.Dropdown(description='X-Axis:', options=self._import_data.columns)
        yaxis_widget = widgets.Dropdown(description='Y-Axis:', options=self._import_data.columns)
        fig = go.FigureWidget(data=[dict(type='scattergl')])
        
        def on_xaxis_change(change):
            if self._ychange is None:
                self._xchange = self._import_data[change['new']]
            else:
                self._xchange = self._import_data[change['new']]
                fig.data[0].x = self._import_data[change['new']]
                fig.data[0].y = self._ychange
        xaxis_widget.observe(on_xaxis_change, names="value")
        
        def on_yaxis_change(change):
            if self._xchange is None:
                self._ychange = self._import_data[change['new']]
            else:
                self._ychange = self._import_data[change['new']]
                fig.data[0].y = self._import_data[change['new']]
                fig.data[0].x = self._xchange
        yaxis_widget.observe(on_yaxis_change, names="value")
        
        self._xchange = None
        self._ychange = None
        
        return widgets.VBox([widgets.HBox([xaxis_widget, yaxis_widget]), fig])

    def _display_preprocess_widgets(self):
        """Display the tab preprocess widgets."""
        tab_contents = ['MakeADate', 'Impute', 'Transform', 'Encode', 'Discretize', 'Standardize']
        children = [self._makeadate(method='POST'), 
                    self._impute(method='POST'), 
                    self._transform(method='POST'), 
                    self._encode(method='POST'), 
                    self._discretize(method='POST'), 
                    self._standardize(method='POST')]
        tab = widgets.Tab()
        tab.children = children
        for i in range(len(children)):
            tab.set_title(i, tab_contents[i])
            tab.children[i].children[-1].on_click(self._run_algo)
        tab.selected_index = None
        
        self._tab = tab
        self._children = children
        self._tab_contents = tab_contents
        
        return tab
        
    def _display_model_widgets(self):
        """"""
        columns_widget = widgets.SelectMultiple(description='Columns:', options=self._import_data.columns)
        return widgets.VBox([columns_widget])
    
    def _display_code_log_widget(self):
        """"""
        train_code_log_widget = widgets.Label(value="""#Train Code Log: \n""")
        dev_code_log_widget = widgets.Label(value="""#Dev Code Log: \n""")
        test_code_log_widget = widgets.Label(value="""#Test Code Log: \n""")
        
        return widgets.VBox([train_code_log_widget, dev_code_log_widget, test_code_log_widget])
         
    def _run_algo(self, btn):
        """Run algorithm user selects on column user selects."""
        clear_output()
        
        if self._tab_contents[self._tab.selected_index] == "MakeADate":
            self._makeadate(method='GET')
        elif self._tab_contents[self._tab.selected_index] == "Impute":
            self._impute(method='GET')
        elif self._tab_contents[self._tab.selected_index] == "Transform":
            self._transform(method='GET')
        elif self._tab_contents[self._tab.selected_index] == "Encode":
            self._encode(method='GET')
        elif self._tab_contents[self._tab.selected_index] == "Discretize":
            self._discretize(method='GET')
        elif self._tab_contents[self._tab.selected_index] == "Standardize":
            self._standardize(method='GET')
            
        self._display_reset_start_widgets()
        self._display_qgrid(new_data=self._import_data)
        self._display_accordion_widget()
        self._export()
        
    def _makeadate(self, method='POST'):
        """"""
        if method == 'POST':
            widget_tab = widgets.HBox([widgets.VBox([widgets.SelectMultiple(description='Columns:', 
                                                                            options=self._import_data.columns),
                                                     widgets.Checkbox(description='Replace columns', value=True)]),
                                       widgets.Button(description='Run Algo', button_style='')])
            return widget_tab
        elif method == 'GET':
            columns = list(self._children[self._tab.selected_index].children[0].children[0].value)
            replace_columns = self._children[self._tab.selected_index].children[0].children[1].value
            try:
                if replace_columns:
                    self._import_data[columns] = self._import_data[columns].apply(pd.to_datetime).add_suffix('_MakeADate')
                else:
                    date_columns = self._import_data[columns]
                    self._import_data = pd.concat([self._import_data, date_columns.apply(pd.to_datetime).add_suffix('_MakeADate')], axis=1)
            except ValueError:
                    self._error_handle("You cannot make one of this/these column(s) a date:", columns)

    def _impute(self, method='POST'):
        """"""
        tab_options = ['Simple', 'Missing']
        if method == 'POST':
            widget_tab = widgets.HBox([widgets.VBox([widgets.Dropdown(description='Algo:', options=tab_options),
                                                     widgets.Dropdown(description='Type:', options=['Mean', 'Median', 'Most_Frequent', 'Constant'])]),
                                       widgets.VBox([widgets.SelectMultiple(description='Columns:', options=self._import_data.columns),
                                                     widgets.Checkbox(description='Replace columns', value=True)]),
                                       widgets.Button(description='Run Algo', button_style='')])
            return widget_tab
        elif method == 'GET':
            algo_type = self._children[self._tab.selected_index].children[0].children[0].value
            strategy_type = self._children[self._tab.selected_index].children[0].children[1].value.lower()
            columns = list(self._children[self._tab.selected_index].children[1].children[0].value)
            replace_columns = self._children[self._tab.selected_index].children[1].children[1].value
            try:
                if replace_columns:
                    if algo_type == 'Simple':
                        column_scaled = SimpleImputer(strategy=strategy_type).fit_transform(self._import_data[columns])
                        self._import_data[columns] = column_scaled
                    elif algo_type == 'Missing':
                        column_scaled = MissingIndicator().fit_transform(self._import_data[columns])
                        self._import_data[columns] = column_scaled
                    self._import_data.rename(columns={column: column+'_'+algo_type for column in columns}, inplace=True)
                else:
                    if algo_type == 'Simple':
                        column_scaled = SimpleImputer(strategy=strategy_type).fit_transform(self._import_data[columns])
                        self._import_data = pd.concat([self._import_data, pd.DataFrame(column_scaled, columns=columns).add_suffix('_Simple')], axis=1)
                    elif algo_type == 'Missing':
                        column_scaled = MissingIndicator().fit_transform(self._import_data[columns])
                        self._import_data = pd.concat([self._import_data, pd.DataFrame(column_scaled, columns=columns).add_suffix('_Missing')], axis=1)
            except AttributeError:
                self._error_handle("Impute can only be used on attributes with numeric data. You cannot Impute on this/these column(s):", columns)

    def _transform(self, method='POST'):
        """"""
        tab_options = ['Quantile', 'Power: Box-Cox', 'Power: Yeo-Johnson']
        if method == 'POST':
            widget_tab = widgets.HBox([widgets.Dropdown(description='Algo:', options=tab_options),
                                       widgets.VBox([widgets.SelectMultiple(description='Columns:', options=self._import_data.columns),
                                                     widgets.Checkbox(description='Replace columns', value=True)]),
                                       widgets.Button(description='Run Algo', button_style='')])
            return widget_tab
        elif method == 'GET':
            algo_type = self._children[self._tab.selected_index].children[0].value
            columns = list(self._children[self._tab.selected_index].children[1].children[0].value)
            replace_columns = self._children[self._tab.selected_index].children[1].children[1].value
            try:
                if replace_columns:
                    if algo_type == 'Quantile':
                        column_scaled = QuantileTransformer(random_state=0).fit_transform(self._import_data[columns])
                        self._import_data[columns] = column_scaled
                    elif algo_type == 'Power: Box-Cox':
                        column_scaled = PowerTransformer(method='box-cox').fit_transform(self._import_data[columns])
                        self._import_data[columns] = column_scaled
                    elif algo_type == 'Power: Yeo-Johnson':
                        column_scaled = PowerTransformer(method='yeo-johnson').fit_transform(self._import_data[columns])
                        self._import_data[columns] = column_scaled
                    self._import_data.rename(columns={column: column+'_'+algo_type for column in columns}, inplace=True)
                else:
                    if algo_type == 'Quantile':
                        column_scaled = QuantileTransformer(random_state=0).fit_transform(self._import_data[columns])
                        self._import_data = pd.concat([self._import_data, pd.DataFrame(column_scaled, columns=columns).add_suffix('_Quantile')], axis=1)
                    elif algo_type == 'Power: Box-Cox':
                        column_scaled = PowerTransformer(method='box-cox').fit_transform(self._import_data[columns])
                        self._import_data = pd.concat([self._import_data, pd.DataFrame(column_scaled, columns=columns).add_suffix('_Power: Box-Cox')], axis=1)
                    elif algo_type == 'Power: Yeo-Johnson':
                        column_scaled = PowerTransformer(method='yeo-johnson').fit_transform(self._import_data[columns])
                        self._import_data = pd.concat([self._import_data, pd.DataFrame(column_scaled, columns=columns).add_suffix('_Power: Yeo-Johnson')], axis=1)
            except ValueError:
                self._error_handle("{} Transform can only be used on attributes with non-negative numeric data. You cannot Transform on this/these column(s):".format(algo_type), columns)
                
    def _encode(self, method='POST'):
        """"""
        tab_options = ['Ordinal', 'OneHot']
        if method == 'POST':
            widget_tab = widgets.HBox([widgets.Dropdown(description='Algo:', options=tab_options),
                                       widgets.VBox([widgets.SelectMultiple(description='Columns:', options=self._import_data.columns),
                                                     widgets.Checkbox(description='Replace columns', value=True)]),
                                       widgets.Button(description='Run Algo', button_style='')])
            return widget_tab
        elif method == 'GET':
                algo_type = self._children[self._tab.selected_index].children[0].value
                columns = list(self._children[self._tab.selected_index].children[1].children[0].value)
                replace_columns = self._children[self._tab.selected_index].children[1].children[1].value
                try:
                    if replace_columns:
                        if algo_type == 'Ordinal':
                            column_scaled = OrdinalEncoder().fit_transform(self._import_data[columns])
                            self._import_data[columns] = column_scaled
                            self._import_data.rename(columns={column: column+'_'+algo_type for column in columns}, inplace=True)
                        elif algo_type == 'OneHot':
                            one_hot = OneHotEncoder(categories='auto').fit(self._import_data[columns])
                            column_scaled = one_hot.transform(self._import_data[columns])
                            self._import_data.drop(columns, inplace=True, axis=1)
                            self._import_data = pd.concat([self._import_data, pd.DataFrame(column_scaled.toarray(), columns=[category for column in one_hot.categories_ for category in column]).add_suffix('_OneHot')], axis=1) 
                    else:
                        if algo_type == 'Ordinal':
                            column_scaled = OrdinalEncoder().fit_transform(self._import_data[columns])
                            self._import_data = pd.concat([self._import_data, pd.DataFrame(column_scaled, columns=columns).add_suffix('_Ordinal')], axis=1)
                        elif algo_type == 'OneHot':
                            one_hot = OneHotEncoder(categories='auto').fit(self._import_data[columns])
                            column_scaled = one_hot.transform(self._import_data[columns])
                            self._import_data.drop(columns, inplace=True, axis=1)
                            self._import_data = pd.concat([self._import_data, pd.DataFrame(column_scaled.toarray(), columns=[category for column in one_hot.categories_ for category in column]).add_suffix('_OneHot')], axis=1) 
                except ValueError:
                    self._error_handle("{} Encode can only be used on data that do not contain NaNs. You cannot Encode on this/these column(s):".format(algo_type), columns)
                
                
    def _discretize(self, method='POST'):
        """"""
        tab_options = ['KBins']
        if method == 'POST':
            widget_tab = widgets.HBox([widgets.VBox([widgets.Dropdown(description='Algo:', options=tab_options),
                                                     widgets.Dropdown(description='Strategy:', options=['Uniform', 'Quantile', 'kMeans']),
                                                     widgets.BoundedIntText(description='NBins:', value=5, min=2, max=1000)]),
                                       widgets.VBox([widgets.SelectMultiple(description='Columns:', options=self._import_data.columns),
                                                     widgets.Checkbox(description='Replace columns', value=True)]),
                                       widgets.Button(description='Run Algo', button_style='')])
            return widget_tab
        elif method == 'GET':
            algo_type = self._children[self._tab.selected_index].children[0].children[0].value
            strategy_type = self._children[self._tab.selected_index].children[0].children[1].value.lower()
            number_bins = self._children[self._tab.selected_index].children[0].children[2].value
            columns = list(self._children[self._tab.selected_index].children[1].children[0].value)
            replace_columns = self._children[self._tab.selected_index].children[1].children[1].value
            try:
                if replace_columns:
                    if algo_type == 'KBins':
                        column_scaled = KBinsDiscretizer(n_bins=number_bins, encode='ordinal',
                                                         strategy=strategy_type).fit_transform(self._import_data[columns])
                        self._import_data[columns] = column_scaled
                    self._import_data.rename(columns={column: column+'_'+algo_type for column in columns}, inplace=True)
                else:
                    if algo_type == 'KBins':
                        column_scaled = KBinsDiscretizer(n_bins=number_bins, encode='ordinal',
                                                         strategy=strategy_type).fit_transform(self._import_data[columns])
                        self._import_data = pd.concat([self._import_data, pd.DataFrame(column_scaled, columns=columns).add_suffix('_KBins')], axis=1)
            except ValueError:
                self._error_handle("{} Discretize can only be used on numeric data that do not contain NaNs. You cannot Discretize on this/these column(s):".format(algo_type), columns)
                
            
    def _standardize(self, method='POST'):
        """"""
        tab_options = ['Standard', 'MinMax', 'MaxAbs', 'Robust']
        if method == 'POST':
            widget_tab = widgets.HBox([widgets.Dropdown(description='Algo:', options=tab_options),
                                       widgets.VBox([widgets.SelectMultiple(description='Columns:', options=self._import_data.columns),
                                                     widgets.Checkbox(description='Replace columns', value=True)]),
                                       widgets.Button(description='Run Algo', button_style='')])
            return widget_tab
        elif method == 'GET':
            algo_type = self._children[self._tab.selected_index].children[0].value
            columns = list(self._children[self._tab.selected_index].children[1].children[0].value)
            replace_columns = self._children[self._tab.selected_index].children[1].children[1].value
            try:
                if replace_columns:
                    if algo_type == 'Standard':
                        column_scaled = StandardScaler().fit_transform(self._import_data[columns])
                        self._import_data[columns] = column_scaled
                    elif algo_type == 'MinMax':
                        column_scaled = MinMaxScaler().fit_transform(self._import_data[columns])
                        self._import_data[columns] = column_scaled
                    elif algo_type == 'MaxAbs':
                        column_scaled = MaxAbsScaler().fit_transform(self._import_data[columns])
                        self._import_data[columns] = column_scaled
                    elif algo_type == 'Robust':
                        column_scaled = RobustScaler().fit_transform(self._import_data[columns])
                        self._import_data[columns] = column_scaled
                    self._import_data.rename(columns={column: column+'_'+algo_type for column in columns}, inplace=True)
                else:
                    if algo_type == 'Standard':
                        column_scaled = StandardScaler().fit_transform(self._import_data[columns])
                        self._import_data = pd.concat([self._import_data, pd.DataFrame(column_scaled, columns=columns).add_suffix('_Standard')], axis=1)
                    elif algo_type == 'MinMax':
                        column_scaled = MinMaxScaler().fit_transform(self._import_data[columns])
                        self._import_data = pd.concat([self._import_data, pd.DataFrame(column_scaled, columns=columns).add_suffix('_MinMax')], axis=1)
                    elif algo_type == 'MaxAbs':
                        column_scaled = MaxAbsScaler().fit_transform(self._import_data[columns])
                        self._import_data = pd.concat([self._import_data, pd.DataFrame(column_scaled, columns=columns).add_suffix('_MaxAbs')], axis=1)
                    elif algo_type == 'Robust':
                        column_scaled = RobustScaler().fit_transform(self._import_data[columns])
                        self._import_data = pd.concat([self._import_data, pd.DataFrame(column_scaled, columns=columns).add_suffix('_Robust')], axis=1)
            except ValueError:
                self._error_handle("{} Standardize can only be used on numeric data. You cannot Standardize on this/these column(s):".format(algo_type), columns)
        
    def _error_handle(self, msg, data):
        """Method to print error msg and start over."""
        clear_output()
        display(Markdown("""<font color="red">**{}**</font>""".format(msg)), data)
        
    def _export(self):
        """Setup widgets for exporting data from QGrid model."""
        export_name_widget = widgets.Text(value="C:\\Users\\username\\export_data.csv",
                                          placeholder='Type something', description='Export File:',
                                          layout=widgets.Layout(width="60%"))
        export_btn_widget = widgets.Button(description='Export Data', button_style='')
        export_btn_widget.on_click(self._run_export)
        display(widgets.HBox([export_name_widget, export_btn_widget]))
        
        self._export_name_widget = export_name_widget
        
    def _run_export(self, btn):
        """Export data from QGrid model."""
        export_data = self._qgrid_data.get_changed_df()
        export_data.to_csv(self._export_name_widget.value)

In [5]:
ManipulateDataAppClass().run()